# Kaggle Notebook (Ex.)
## Data Science Bowl 2019/2020

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

np.random.seed(42)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Reading in Data

You can't just read in the new test/train csvs in the kaggle kernel, you have to run the whole feature engineering script on the original files.

In [2]:
train = pd.read_csv('./data/new_train2.csv')
test = pd.read_csv('./data/new_test2.csv')
labels = pd.read_csv('./data/train_labels.csv')

#### Comparing Train and Test

In [3]:
# filtering out any columns not in test
test_cols = test.columns
traint = train[test_cols]

#### Merging Train and Train Labels

In [4]:
acc = labels[['game_session', 'accuracy_group']]
trainlab = traint.merge(acc, on = 'game_session')

#### Dropping Columns

In [5]:
trainlab.drop(columns = ['start_game'], inplace = True)
test.drop(columns = ['start_game'], inplace = True)

### Filling Nulls

In [6]:
trainlab.fillna(9999, inplace = True)
test.fillna(9999, inplace = True)

### Set X and y

In [7]:
X = trainlab.drop(columns = ['accuracy_group', 'game_session', 'installation_id'])
y = trainlab['accuracy_group']

test_test = test.drop(columns = ['game_session', 'installation_id'])

### Scaling

In [9]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)
test_ss = ss.transform(test_test)

In [10]:
test_ss = ss.transform(test_test)

### Model

In [11]:
rf = RandomForestClassifier(max_depth = 10,
                            n_estimators = 1000, 
                            n_jobs = -2, 
                            oob_score = False, 
                            warm_start = True)
rf.fit(X_train_ss, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-2,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [12]:
preds = rf.predict(test_ss)

In [20]:
x = {'installation_id': test['installation_id'],
     'accuracy_group': preds}
sample = pd.DataFrame(x)

In [16]:
sample.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [21]:
sample.to_csv('./submission.csv', index = False)